In [166]:
# Импортируем библиотеки
import pandas as pd
import datetime
from helpful import *
import matplotlib.pyplot as plt
import seaborn as sns

In [167]:
# Считываем данные из файла
df = pd.read_excel('data.xlsx')

In [168]:
# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [169]:
# Удаляем месяц, так как он не несет никакой информации
df = df.drop('month', axis=1)

# Удалим строки с merchant_name, которое встречается меньше 6 раз в df
df = df.groupby('merchant_name').filter(lambda x : len(x) > 5)
df.head(200)

,merchant_name,day,cashback
8350,#SEKTA,2022-04-18 00:00:00,347.49
8351,#SEKTA,2022-04-19 00:00:00,366.19
8352,#SEKTA,2022-04-20 00:00:00,142.50
8353,#SEKTA,2022-04-21 00:00:00,222.66
8354,#SEKTA,2022-04-22 00:00:00,90.23
...,...,...,...
9398,1С Интерес,2022-05-12 00:00:00,822.69
9399,1С Интерес,2022-05-13 00:00:00,693.33
9400,1С Интерес,2022-05-14 00:00:00,1419.64
9401,1С Интерес,2022-05-15 00:00:00,770.77


In [170]:
# Переведем столбец day формата "YYYY-MM-DD hh:mm:ss" в столбец с результатом get_date
df['day'] = df['day'].apply(get_date)
df['weekday'] = df['day'].apply(get_weekday)
df['is_weekend'] = df['weekday'].apply(lambda x : 1 if x in [5, 6] else 0)

In [171]:
df

,merchant_name,day,cashback,weekday,is_weekend
8350,#SEKTA,2022-04-18,347.49,0,0
8351,#SEKTA,2022-04-19,366.19,1,0
8352,#SEKTA,2022-04-20,142.50,2,0
8353,#SEKTA,2022-04-21,222.66,3,0
8354,#SEKTA,2022-04-22,90.23,4,0
...,...,...,...,...,...
13969,интернет-магазин Технопарк,2022-07-27,26123.62,2,0
13970,интернет-магазин Технопарк,2022-07-28,25921.13,3,0
13971,интернет-магазин Технопарк,2022-07-29,35976.24,4,0
13972,интернет-магазин Технопарк,2022-07-30,32105.00,5,1


In [172]:
names = df['merchant_name'].unique()
for i in names:
    print(list(names).index(i))
    d_obj = get_date(datetime.datetime(2022,1,1))
    while d_obj < get_date(datetime.datetime(2023,2,1)):
        if not(d_obj in df[df.merchant_name == i]['day'].values):
            df = df._append({
            'merchant_name' : i, 
            'day' : d_obj,
            'cashback' : 0, 
            'weekday' : get_weekday(get_next_day(d_obj)),
            'is_weekend' : 1 if get_weekday(d_obj) in [5, 6] else 0},
            ignore_index=True)
        d_obj = get_next_day(d_obj)
df = df.sort_values(by=['merchant_name', 'day'])
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,merchant_name,day,cashback,weekday,is_weekend
27406,#SEKTA,2022-01-01,0.0,6,1
27407,#SEKTA,2022-01-02,0.0,0,1
27408,#SEKTA,2022-01-03,0.0,1,0
27409,#SEKTA,2022-01-04,0.0,2,0
27410,#SEKTA,2022-01-05,0.0,3,0
...,...,...,...,...,...
182500,интернет-магазин Технопарк,2023-01-27,0.0,5,0
182501,интернет-магазин Технопарк,2023-01-28,0.0,6,1
182502,интернет-магазин Технопарк,2023-01-29,0.0,0,1
182503,интернет-магазин Технопарк,2023-01-30,0.0,1,0


In [173]:
'''
# Заполним пропуски в столбце cashback средним значением
df['cashback'] = df['cashback'].fillna(df['cashback'].mean())

# Если между двумя строками разница 2 дня, то добавим строку между ними с пропущенными значениями
for i in range(len(df) - 1):
    if get_days_diff(df.iloc[i]['day'], df.iloc[i + 1]['day']) == 2:
        df = df._append({
            'merchant_name' : df.iloc[i]['merchant_name'], 
            'day' : get_next_day(df.iloc[i]['day']),
            'cashback' : (df.iloc[i]['cashback'] + df.iloc[i + 1]['cashback']) / 2, 
            'weekday' : get_weekday(get_next_day(df.iloc[i]['day'])),
            'is_weekend' : 1 if get_weekday(get_next_day(df.iloc[i]['day'])) in [5, 6] else 0},
            ignore_index=True)

# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])'''

"\n# Заполним пропуски в столбце cashback средним значением\ndf['cashback'] = df['cashback'].fillna(df['cashback'].mean())\n\n# Если между двумя строками разница 2 дня, то добавим строку между ними с пропущенными значениями\nfor i in range(len(df) - 1):\n    if get_days_diff(df.iloc[i]['day'], df.iloc[i + 1]['day']) == 2:\n        df = df._append({\n            'merchant_name' : df.iloc[i]['merchant_name'], \n            'day' : get_next_day(df.iloc[i]['day']),\n            'cashback' : (df.iloc[i]['cashback'] + df.iloc[i + 1]['cashback']) / 2, \n            'weekday' : get_weekday(get_next_day(df.iloc[i]['day'])),\n            'is_weekend' : 1 if get_weekday(get_next_day(df.iloc[i]['day'])) in [5, 6] else 0},\n            ignore_index=True)\n\n# Отсортируем данные по именам и возрастании даты\ndf = df.sort_values(by=['merchant_name', 'day'])"

In [174]:
# Для каждого значения merchant_name построим график cashback
'''
for merchant_name in df['merchant_name'].unique():
    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')
    plt.title(merchant_name)
    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')
    plt.close()
'''

"\nfor merchant_name in df['merchant_name'].unique():\n    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')\n    plt.title(merchant_name)\n    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')\n    plt.close()\n"

In [175]:
# Сохраним df как clear_data.xlsx  

df.to_excel('Книга1.xlsx', index=False) 


"\ndf.to_excel('clear_data.xlsx', index=False) \n"

In [176]:
df

,merchant_name,day,cashback,weekday,is_weekend
27406,#SEKTA,2022-01-01,0.0,6,1
27407,#SEKTA,2022-01-02,0.0,0,1
27408,#SEKTA,2022-01-03,0.0,1,0
27409,#SEKTA,2022-01-04,0.0,2,0
27410,#SEKTA,2022-01-05,0.0,3,0
...,...,...,...,...,...
182500,интернет-магазин Технопарк,2023-01-27,0.0,5,0
182501,интернет-магазин Технопарк,2023-01-28,0.0,6,1
182502,интернет-магазин Технопарк,2023-01-29,0.0,0,1
182503,интернет-магазин Технопарк,2023-01-30,0.0,1,0
